This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'o'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
response = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2020-07-23&end_date=2020-07-24&api_key='+API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
response.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-07-10) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-07-24T22:16:22.029Z',
  'newest_available_date': '2020-07-24',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2020-07-23',
  'end_date': '2020-07-24',
  'data': [['2020-07-24',
    90.5,
    90.5,
    87.85,
    87.9,
    None,
    177031.0,
    15683394.95,
    None,
    None,
    None],
   ['2020-07-23',
    88.4,
    91.55,
    88.15,
    91.55,
    None

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#We'll start by defining the start and end dates then request the corresponding data from Quandl
startDate = '2017-01-01'
endDate = '2017-12-31'
response = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date='+startDate+'&end_date='+ endDate + '&api_key='+API_KEY)
#Convert the json to a dict:
data = response.json()
#lets take a look at the keys
data['dataset'].keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

In [6]:
# Next we'll create a new dictionary that will be much more streamlined for the analysis. It will be a nested dictionary with 
#the first set of keys being the date and the value for wach date will be dictionaries with keys that correspond to the column_names
#from the json date and their corresponding values as the values. Note that this dictionary would be structred to seamlessly be
#converted to a pandas dataframe (if we were allowed to use one in this exercise...)

#First initialize an empty dictionary
stocks = {}
#for each of the dates in the data set (first key from the json), we'll create an empty dictionary
for day in data['dataset']['data']:
    stocks[day[0]] = {}
    #Next, we add the key value pairs from the 'column_names' nested dict from the json data and the day dictionary (this is the 
    #dictionary that contains the corresponding data), note that the 0th element of the column_names value and the day value
    #is the date itself so we need to skip that
    for num, col in enumerate(data['dataset']['column_names'][1::]):
        stocks[day[0]][col] = day[num+1]

In [10]:
#We need to drop any days for which we may be missing the high, low or closing data. I'll accomplish this by creating a list of days to delete
missingData = []
[missingData.append(day) for day in stocks if not stocks[day]['High'] or not stocks[day]['Low'] or not stocks[day]['Close']]
        
#Now we need to delete the days that are missing data

for day in missingData:
    del stocks[day] 

[]

In [11]:
#determine the high and low closing values using list comprehension
highClose = max(day['Close'] for day in stocks.values())
print('The highest closing price during 2017 was: ' + str(highClose))
lowClose = min(day['Close'] for day in stocks.values())
print('The lowest closing price during 2017 was : ' + str(lowClose))

The highest closing price during 2017 was: 53.09
The lowest closing price during 2017 was : 34.06


In [12]:
#Determine the largest change in one day using list comprehension
maxChange = max([day['High']-day['Low'] for day in stocks.values()])
print('The largest intraday change was: ' + str(maxChange))


The largest intraday change was: 2.8100000000000023


In [13]:
#Determine the largest interday change. To do this we'll create a list of the dates for which we have data, then iterate
#through the list of dates subtracting the last close from the current days close. 
#First create the list of days.
days = list(stocks.keys())
#initialize a list for the interDay change
interDay = []
#Append the change from the current close to the last days close for each day
[interDay.append(stocks[days[day]]['Close'] - stocks[days[day-1]]['Close']) for day in range(1, len(days))]
# We want to capture the largest swing regarless of direction so if the absolute value of themin (meaning a negative swing) is 
#greater than the max swing, we'll report the minimum as the largest swing. 
minInterDay = min(interDay)
maxInterDay = max(interDay)
if abs(minInterDay) > maxInterDay:
    maxInterDay = minInterday
print('The maximum change between any two days during 2017 was: ' + str(maxInterDay))

The maximum change between any two days during 2017 was: 2.559999999999995


In [14]:
#To find the average volume, we'll create a list of the daily trade volumes, then calculate the mean of the list
#initialize an empty list
dailyVol = []
#Append each days traded volume value
[dailyVol.append(day['Traded Volume']) for day in stocks.values()]
#Calculate the mean
meanVol = float(sum(dailyVol)/len(dailyVol))
print('The average daily traded volume for 2017 was: ' + str(meanVol))

The average daily traded volume for 2017 was: 89124.33725490196


In [15]:
#I'll write a short function to return the median of a list 

def medianStock(stockData):
    #first we'll sort the stock data 
    stockData = sorted(dailyVol)
    #Next we need to determine if the number of elements is even or odd
    #If its odd then we just find the mid-point 
    if len(stockData)%2 != 0:
        mid = round(len(stockData)/2)
        return stockData[mid]
    else:
        #If the number of elements is even, we'll need ot find the average of the two midpoints
        mid = len(stockData)/2
        return (stockData[mid+1] + stockData[mid-1])/2

print('The median stock volume for 2017 was: ' + str(medianStock(dailyVol)))


The median stock volume for 2017 was: 76600.0
